In [1]:
import pandas as pd
import csv
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

##### Load all the data into suitable dataframe using pandas

In [2]:
clinical_data = pd.read_csv("./survival_data.csv")
flair_data = pd.read_csv("./TrainingDataHGGFLAIR.csv")
t1ce_data = pd.read_csv("./TrainingDataHGGT1CE.csv")

In [3]:
merge_data1 = pd.merge(clinical_data, flair_data, on='folder_name')
merge_data2 = pd.merge(merge_data1, t1ce_data, on='folder_name')
#merge_data2.to_csv("./new csvs/preprocessed_dataset.csv")

##### Extract integer and float datatype features

In [4]:
float_var = merge_data2.dtypes[merge_data2.dtypes=='float'].index.values
int_var = merge_data2.dtypes[merge_data2.dtypes=='int'].index.values

f_list = [x for x in float_var]
i_list = [x for x in int_var]
use_features = f_list + i_list
print(len(use_features))

data_used = merge_data2[use_features]

212


In [6]:
#data_used.to_csv(("./new csvs/use_features_dataset.csv"))

Normalising data using MinMaxScaler()

min_max_scaler = preprocessing.MinMaxScaler()
data_normalised = pd.DataFrame(min_max_scaler.fit_transform(data_used), columns=data_used[use_features].columns, index=data_used[use_features].index)
#data_normalised.to_csv("./new csvs/normalised_dataset.csv")

label = 'Survival'
features = [x for x in use_features if x not in label]
b = data_normalised[label]  #data_labels
a = data_normalised[features]
a_train, a_test, b_train, b_test = train_test_split(a, b, test_size=0.30)

clf = svm.SVR(kernel='rbf')
clf.fit(a_train, b_train)

predicted = clf.predict(a_test)
print(r2_score(b_test,predicted))

from sklearn.ensemble import RandomForestRegressor
rFG = RandomForestRegressor(max_depth = 2, random_state=0, n_estimators = 100)
rFG.fit(a_train, b_train)

predicted_rfg = clf.predict(a_test)
print(r2_score(b_test,predicted_rfg))

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(a_train, b_train)

predicted_lr1 = lr.predict(a_test)
print(r2_score(b_test,predicted_lr1))

In [14]:
"""
from sklearn.feature_selection import SelectKBest, f_regression
data_used.shape
test = SelectKBest(score_func=f_regression, k=50)
fit = test.fit(a_train, b_train)
np.set_printoptions(precision=3)
#print(fit.scores_)

features_ft = fit.transform(a_train)
print(features_ft[0:5])


"""

'\nfrom sklearn.feature_selection import SelectKBest, f_regression\ndata_used.shape\ntest = SelectKBest(score_func=f_regression, k=50)\nfit = test.fit(a_train, b_train)\nnp.set_printoptions(precision=3)\n#print(fit.scores_)\n\nfeatures_ft = fit.transform(a_train)\nprint(features_ft[0:5])\n\n\n'

Rank the features for their importance

from sklearn.feature_selection import RFE
#selector = RFE(lr, 212, 1, 0)     #for linear regression
selector = RFE(rFG, 212, 1, 0)
fitting = selector.fit(a_train, b_train)

selected_f = fitting.support_
all_f = a_train.keys()
rank_f = fitting.ranking_

b = {}

for i in range(0, len(selected_f)):
    #print(selected_f[i])
    if(selected_f[i]==True) :
        b[all_f[i]]=rank_f[i]

z score normalistation

0 mean and unit standard deviation

svm non linear

rank the features

caclulate all the metrics as in the paper

cross validation 5 fold and 10 fold

report results for both of the above versions

# Normalisation using Standard Scaler

##### Normalising 'data_used' dataframe using standardScaler()

In [5]:
scaler = preprocessing.StandardScaler()
data_standardised = pd.DataFrame(scaler.fit_transform(data_used), columns=data_used[use_features].columns, index=data_used[use_features].index)

In [15]:
#data_standardised.to_csv("Standardised_data.csv")

Splitting the dataset into train and test features = (newa_train, newa_test)  labels = (newb_train, newb_test)

In [6]:
label = 'Survival'
features = [x for x in use_features if x not in label]

In [16]:
new_b = data_standardised[label]  #data_labels
new_a = data_standardised[features]
#newa_train, newa_test, newb_train, newb_test = train_test_split(new_a, new_b, test_size=0.20)

Importing svm.SVR()

In [17]:
from sklearn import svm
clf = svm.SVR(kernel='poly')

# Cross Validation (CV)

In [18]:
from sklearn.model_selection import cross_val_score

5 Fold CV

In [19]:
median_SE_5 = cross_val_score(clf, new_a, new_b, scoring = 'neg_median_absolute_error', cv=5)
MSE_5 = cross_val_score(clf, new_a, new_b, scoring = 'neg_mean_squared_error', cv=5)
explained_variance_5 = cross_val_score(clf, new_a, new_b, scoring = 'explained_variance', cv=5)

Visualising metrics of 5 fold CV

In [20]:
print('\n 5 Fold CV results \n')

for i in range(0,5) :
    print(i+1,' - fold\'s ','median_se= ',median_SE_5[i],' mse= ', MSE_5[i],' expl_var= ',explained_variance_5[i])


 5 Fold CV results 

(1, " - fold's ", 'median_se= ', -0.6228583465154689, ' mse= ', -1.992738391335075, ' expl_var= ', -1.3250911048390122)
(2, " - fold's ", 'median_se= ', -0.7122153150671409, ' mse= ', -2.140214831669809, ' expl_var= ', -0.16414118812059897)
(3, " - fold's ", 'median_se= ', -0.5240161346636534, ' mse= ', -1.1408091259739441, ' expl_var= ', -0.44973264924839573)
(4, " - fold's ", 'median_se= ', -0.6081387032202278, ' mse= ', -1.1193101505209784, ' expl_var= ', 0.06764903897335472)
(5, " - fold's ", 'median_se= ', -0.40599081493109446, ' mse= ', -1.004456145549448, ' expl_var= ', -0.5518747022299224)


In [21]:
"""
names = [0,1,2,3,4]
data = np.array([names, median_SE_5, MSE_5, explained_variance_5])
print(pd.DataFrame(data=data[1:,1:], index=data[1:,0], columns=data[0,1:]))
print(median_SE_5)
"""

'\nnames = [0,1,2,3,4]\ndata = np.array([names, median_SE_5, MSE_5, explained_variance_5])\nprint(pd.DataFrame(data=data[1:,1:], index=data[1:,0], columns=data[0,1:]))\nprint(median_SE_5)\n'

10 Fold CV

In [22]:
median_SE_10 = cross_val_score(clf, new_a, new_b, scoring = 'neg_median_absolute_error', cv=10)
MSE_10 = cross_val_score(clf, new_a, new_b, scoring = 'neg_mean_squared_error', cv=10)
explained_variance_10 = cross_val_score(clf, new_a, new_b, scoring = 'explained_variance', cv=10)

Visualising metrics of 10 fold CV

In [23]:
print('\n 10-Fold CV results \n')

for i in range(0,10) :
    print(i+1,' - fold\'s ','median_se= ',median_SE_10[i],' mse= ', MSE_10[i],' expl_var= ',explained_variance_10[i])


 10-Fold CV results 

(1, " - fold's ", 'median_se= ', -0.6245513356477137, ' mse= ', -2.7936863159641754, ' expl_var= ', -1.6245373025552388)
(2, " - fold's ", 'median_se= ', -0.444578811805348, ' mse= ', -0.8408877974864971, ' expl_var= ', 0.08249786324493902)
(3, " - fold's ", 'median_se= ', -0.6898435431192433, ' mse= ', -1.4982027528089334, ' expl_var= ', -0.08664065230384099)
(4, " - fold's ", 'median_se= ', -1.2342987836413526, ' mse= ', -2.5209530744085713, ' expl_var= ', -0.0551968690863629)
(5, " - fold's ", 'median_se= ', -0.3767825854855964, ' mse= ', -1.0822786985743569, ' expl_var= ', -2.5208708333294267)
(6, " - fold's ", 'median_se= ', -0.5542973896541944, ' mse= ', -1.2987493149820364, ' expl_var= ', -0.016354490503409203)
(7, " - fold's ", 'median_se= ', -0.4573685225108183, ' mse= ', -1.2087436490293366, ' expl_var= ', 0.19422338974331488)
(8, " - fold's ", 'median_se= ', -0.7385877075150559, ' mse= ', -0.9448893158693521, ' expl_var= ', -0.00439536835770693)
(9, " 